In [1]:
import numpy as np

# Function to print the game board
def print_board(board):
    for row in board:
        print(" | ".join(row))
        print("-" * 9)

# Function to check for a win
def check_win(board, player):
    # Check rows, columns, and diagonals
    return (
        any(np.all(board[i, :] == player) for i in range(3)) or  # Check rows
        any(np.all(board[:, j] == player) for j in range(3)) or  # Check columns
        np.all(np.diag(board) == player) or                     # Check main diagonal
        np.all(np.diag(np.fliplr(board)) == player)             # Check anti-diagonal
    )

# Function to check for a draw
def check_draw(board):
    return not np.any(board == " ")

# Main function to play the game
def play_game():
    # Initialize the board with empty spaces using a NumPy array
    board = np.full((3, 3), " ")
    players = ["X", "O"]
    current_player = players[0]

    # Game loop
    while True:
        print_board(board)
        print(f"Player {current_player}'s turn.")
        
        # Get valid input from the player
        try:
            row = int(input("Enter row (0, 1, or 2): "))
            col = int(input("Enter column (0, 1, or 2): "))
            if board[row, col] != " ":
                print("Cell already occupied, try again.")
                continue
        except (ValueError, IndexError):
            print("Invalid input, please enter numbers between 0 and 2.")
            continue

        # Place the player's mark on the board
        board[row, col] = current_player

        # Check for a win or a draw
        if check_win(board, current_player):
            print_board(board)
            print(f"Player {current_player} wins!")
            break
        if check_draw(board):
            print_board(board)
            print("It's a draw!")
            break

        # Switch players
        current_player = players[1] if current_player == players[0] else players[0]

# Start the game
play_game()


  |   |  
---------
  |   |  
---------
  |   |  
---------
Player X's turn.
  | X |  
---------
  |   |  
---------
  |   |  
---------
Player O's turn.
  | X |  
---------
  |   |  
---------
  | O |  
---------
Player X's turn.
Cell already occupied, try again.
  | X |  
---------
  |   |  
---------
  | O |  
---------
Player X's turn.
X | X |  
---------
  |   |  
---------
  | O |  
---------
Player O's turn.
X | X |  
---------
O |   |  
---------
  | O |  
---------
Player X's turn.
X | X |  
---------
O | X |  
---------
  | O |  
---------
Player O's turn.
X | X |  
---------
O | X |  
---------
O | O |  
---------
Player X's turn.
X | X |  
---------
O | X | X
---------
O | O |  
---------
Player O's turn.
X | X |  
---------
O | X | X
---------
O | O | O
---------
Player O wins!


In [5]:
import numpy as np
import heapq

GOAL_STATE = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
])

def find_blank_position(state):
    return np.argwhere(state == 0)[0]

def manhattan_distance(state):
    distance = 0
    for i in range(3):
        for j in range(3):
            value = state[i, j]
            if value != 0:
                target_x, target_y = divmod(value - 1, 3)
                distance += abs(i - target_x) + abs(j - target_y)
    return distance

def get_possible_moves(state):
    moves = []
    x, y = find_blank_position(state)
    directions = {
        'up': (x - 1, y),
        'down': (x + 1, y),
        'left': (x, y - 1),
        'right': (x, y + 1)
    }
    
    for direction, (new_x, new_y) in directions.items():
        if 0 <= new_x < 3 and 0 <= new_y < 3:
            new_state = state.copy()
            new_state[x, y], new_state[new_x, new_y] = new_state[new_x, new_y], new_state[x, y]
            moves.append(new_state)
    
    return moves

def a_star(initial_state):
    visited = set()
    priority_queue = []
    heapq.heappush(priority_queue, (0, tuple(map(tuple, initial_state)), 0, []))

    while priority_queue:
        _, current_state_tuple, depth, path = heapq.heappop(priority_queue)
        current_state = np.array(current_state_tuple)

        if np.array_equal(current_state, GOAL_STATE):
            return path + [current_state]

        if current_state_tuple in visited:
            continue
        visited.add(current_state_tuple)

        for next_state in get_possible_moves(current_state):
            next_state_tuple = tuple(map(tuple, next_state))
            if next_state_tuple not in visited:
                new_path = path + [current_state]
                cost = depth + 1 + manhattan_distance(next_state)
                heapq.heappush(priority_queue, (cost, next_state_tuple, depth + 1, new_path))

    return None

def print_puzzle(state):
    for row in state:
        print(" ".join(str(x) if x != 0 else " " for x in row))
    print("")

def solve_puzzle(initial_state):
    print("Initial State:")
    print_puzzle(initial_state)
    
    path = a_star(initial_state)
    
    if path:
        print("Solution found!")
        for step, state in enumerate(path):
            print(f"Step {step}:")
            print_puzzle(state)
        print("Goal Reached!")
    else:
        print("No solution found.")

initial_state = np.array([
    [1, 2, 3],
    [4, 0, 5],
    [7, 8, 6]
])

solve_puzzle(initial_state)



Initial State:
1 2 3
4   5
7 8 6

Solution found!
Step 0:
1 2 3
4   5
7 8 6

Step 1:
1 2 3
4 5  
7 8 6

Step 2:
1 2 3
4 5 6
7 8  

Goal Reached!


In [8]:
def travelling_salesman_dp(distance_matrix):
    n = len(distance_matrix)
    memo = {}

    def tsp(current, visited):
        if visited == (1 << n) - 1:
            return distance_matrix[current][0]
        
        if (current, visited) in memo:
            return memo[(current, visited)]
        
        min_cost = float('inf')
        for next_city in range(n):
            if visited & (1 << next_city) == 0:
                cost = distance_matrix[current][next_city] + tsp(next_city, visited | (1 << next_city))
                min_cost = min(min_cost, cost)
        
        memo[(current, visited)] = min_cost
        return min_cost
    
    return tsp(0, 1 << 0)

# Example usage
distance_matrix = [
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
]

min_cost = travelling_salesman_dp(distance_matrix)
print("Minimum path cost:", min_cost)


Minimum path cost: 80


In [10]:
from collections import deque

def water_jug_problem(jug1_capacity, jug2_capacity, target):
    initial_state = (0, 0)
    queue = deque([(initial_state, [])])
    visited = set()
    visited.add(initial_state)
    
    while queue:
        (jug1, jug2), path = queue.popleft()
        
        if jug1 == target or jug2 == target:
            return path + [(jug1, jug2)]
        
        possible_states = [
            (jug1_capacity, jug2),                    
            (jug1, jug2_capacity),                    
            (0, jug2),                                
            (jug1, 0),                                
            (min(jug1 + jug2, jug1_capacity),         
             max(0, jug2 - (jug1_capacity - jug1))),
            (max(0, jug1 - (jug2_capacity - jug2)),   
             min(jug1 + jug2, jug2_capacity))
        ]
        
        for state in possible_states:
            if state not in visited:
                visited.add(state)
                queue.append((state, path + [(jug1, jug2)]))
    
    return []  # Return empty list if no solution found

# Test Case 1: Jug 1 - 3 litres, Jug 2 - 5 litres, Measure - 4 litres
jug1_capacity = 3
jug2_capacity = 5
target = 4
result1 = water_jug_problem(jug1_capacity, jug2_capacity, target)
print("Solution for Jug 1 - 3 litres, Jug 2 - 5 litres, Measure - 4 litres:")
if result1:
    for state in result1:
        print(state)
else:
    print("No solution found.")

# Test Case 2: Jug 1 - 6 litres, Jug 2 - 2 litres, Measure - 3 litres
jug1_capacity = 6
jug2_capacity = 2
target = 3
result2 = water_jug_problem(jug1_capacity, jug2_capacity, target)
print("\nSolution for Jug 1 - 6 litres, Jug 2 - 2 litres, Measure - 3 litres:")
if result2:
    for state in result2:
        print(state)
else:
    print("No solution found.")


Solution for Jug 1 - 3 litres, Jug 2 - 5 litres, Measure - 4 litres:
(0, 0)
(0, 5)
(3, 2)
(0, 2)
(2, 0)
(2, 5)
(3, 4)

Solution for Jug 1 - 6 litres, Jug 2 - 2 litres, Measure - 3 litres:
No solution found.
